<a href="https://colab.research.google.com/github/stayup24h/Hangul-to-Unicode-Obfuscation-Project/blob/main/model_building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# initial
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, LSTM, Dense, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
#모델 설계

def model(input_shape, num_classes):
    model = Sequential()

    # CNN layers
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))

    # Reshape for RNN layers
    new_height = input_shape[0] // 8
    new_width = input_shape[1] // 8
    model.add(Reshape(target_shape=(new_width, new_height * 128)))

    # RNN layers (Bidirectional LSTM)
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Bidirectional(LSTM(64, return_sequences=True)))

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    return model

In [ ]:
# 모델 체크포인트 관리 클래스

checkpoint_path = "training_checkpoints/epoch_{epoch:04d}/model.weights.h5"

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_freq='epoch'
)

class CheckpointManager:
    def __init__(self, checkpoint_path_template, max_recent=3, extra_interval=50):
        self.checkpoint_path_template = checkpoint_path_template
        self.max_recent = max_recent
        self.extra_interval = extra_interval
        self.recent_epochs = []

    def load(self, model, epoch, checkpoint_path_template=checkpoint_path):
        path = checkpoint_path_template.format(epoch=epoch)
        if os.path.exists(path):
            model.load_weights(path)
            print(f"Loaded weights from {path}")
            return True
        else:
            print(f"No checkpoint found at {path}")
            return False

    def save(self, model, epoch):
        # Always save at extra_interval epochs
        if epoch % self.extra_interval == 0 and epoch not in self.recent_epochs:
            path = self.checkpoint_path_template.format(epoch=epoch)
            model.save_weights(path)
            print(f"Extra checkpoint saved for epoch {epoch} at {path}")

        # Save recent checkpoints
        self.recent_epochs.append(epoch)
        if len(self.recent_epochs) > self.max_recent:
            # Remove oldest checkpoint from recent
            old_epoch = self.recent_epochs.pop(0)
            old_path = self.checkpoint_path_template.format(epoch=old_epoch)
            if os.path.exists(old_path):
                os.remove(old_path)
                print(f"Removed old checkpoint at {old_path}")

        path = self.checkpoint_path_template.format(epoch=epoch)
        model.save_weights(path)
        print(f"Checkpoint saved for epoch {epoch} at {path}")